**1. Importamos las librerias que vamos a emplear**

In [11]:
# pip install requirement.txt
import pandas as pd
import plotly_express as px
import streamlit as st

**2. importamos el DataFrame**

In [12]:
# Ruta pc casa
#vehicles = pd.read_csv(r"C:\Users\jorge\OneDrive\Escritorio\DATA-SCIENTIST\DATA_SCIENTIS_TRIPLETEN\Sprint 7\Proyecto-SP7\vehicles_us.csv", sep = ',')

#ruta pc oficina
vehicles = pd.read_csv(r"C:\Users\Usuario\Desktop\DATA SCIENTIST\DATA_SCIENTIS_TRIPLETEN\Sprint 7\Proyecto-SP7\vehicles_us.csv", sep = ',')

**3. Análisis exploratorio**

In [13]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [14]:
vehicles.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


**Observaciones:**
- Valores nulos en las columnas : model_year, cylinders, odometer, paint_color e is_4wd.
- is_4wd es float64, lo que indica que probablemente solo tenga valores 1.0 cuando el vehículo es 4WD y NaN cuando no lo es.
- date_posted está en formato object, sería ideal convertirlo a datetime para facilitar análisis temporales.


**Limpieza de datos**

Para limpiar el DataFrame y no obtener problemas o informacion incorrecta a partir del analisis a desarrollar pordemos emplear el siguiente enfoque.

- Valores nulos:
    - rellenar los valores faltantes en model_year, cylinders, odometer y paint_color.
    - Remplazar los datos NaN en is_4wd por 0.
- Corregir los tipos de datos que lo requieran:
    - Convertir date_posted a formato datetime


**Valores nulos**

El enfoque que se dara para el tratamiento de datos nulos varia dependiendo el contenido de cada columna, a continuacion presento cual será la metodologia apolicada para cada caso. 

- 'model_year': Se rellenaran los valores nulos con la mediana, de este modo se mantiene la coherencia.
- 'cylinders': Rellenar con la moda, se empleará el dato más comun por modelo de auto.
- 'odometer': Posible solucion: llenar con la mediana asociada al modelo.
- 'paint_color': dejar vasio o rellenar con 'unknown'.
- 'is_4wd': Remplazar NaN por 0.


In [15]:
# # Manejo de valores nulos

# Rellenar model_year con la mediana por modelo de auto
vehicles["model_year"] = vehicles.groupby("model")["model_year"].transform(lambda x: x.fillna(x.median()))

# Rellenar cylinders con la moda por modelo de auto
vehicles["cylinders"] = vehicles.groupby("model")["cylinders"].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))

# Rellenar odometer con la mediana por modelo y año del auto
vehicles["odometer"] = vehicles.groupby(["model", "model_year"])["odometer"].transform(lambda x: x.fillna(x.median()))

# Reemplazar NaN en paint_color con "unknown"
vehicles["paint_color"].fillna("unknown", inplace=True)

# 5Convertir NaN en is_4wd a 0
vehicles["is_4wd"].fillna(0, inplace=True)

# Verificar si aún hay valores nulos
vehicles.isnull().sum()


C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1241: Runti

price            0
model_year       0
model            0
condition        0
cylinders        0
fuel             0
odometer        83
transmission     0
type             0
paint_color      0
is_4wd           0
date_posted      0
days_listed      0
dtype: int64

In [16]:
# Eliminar filas donde 'odometer' sigue siendo nulo
vehicles.dropna(subset=["odometer"], inplace=True)

# Verificar si quedan valores nulos en el dataset
vehicles.isnull().sum()


price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

In [17]:
vehicles.head()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,unknown,1.0,2018-06-23,19
1,25500,2011.0,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,177500.0,automatic,pickup,unknown,0.0,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28


**Correccion tipo de datos**



In [18]:
# Convertir date_posted a datetime
vehicles["date_posted"] = pd.to_datetime(vehicles["date_posted"])

# Convertir model_year y cylinders a enteros
vehicles["model_year"] = vehicles["model_year"].astype(int)
vehicles["cylinders"] = vehicles["cylinders"].astype(int)
vehicles["is_4wd"] = vehicles["is_4wd"].astype(int)

# Verificar los cambios
vehicles.info()


<class 'pandas.core.frame.DataFrame'>
Index: 51442 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51442 non-null  int64         
 1   model_year    51442 non-null  int64         
 2   model         51442 non-null  object        
 3   condition     51442 non-null  object        
 4   cylinders     51442 non-null  int64         
 5   fuel          51442 non-null  object        
 6   odometer      51442 non-null  float64       
 7   transmission  51442 non-null  object        
 8   type          51442 non-null  object        
 9   paint_color   51442 non-null  object        
 10  is_4wd        51442 non-null  int64         
 11  date_posted   51442 non-null  datetime64[ns]
 12  days_listed   51442 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 5.5+ MB


Ahora el Dataframe no tiene valores nulos y cuenta con los tipos de datos adecuados para cada variable

****


In [19]:
vehicles.to_csv("vehicles_cleaned.csv", index=False)
